In [1]:
!git clone git@github.com:andrewyng/translation-agent.git

Cloning into 'translation-agent'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 226 (delta 48), reused 41 (delta 31), pack-reused 151
Receiving objects: 100% (226/226), 26.82 MiB | 4.63 MiB/s, done.
Resolving deltas: 100% (87/87), done.


## prompt & workflow

### onechunk


- `one_chunk_initial_translation`
- `one_chunk_reflect_on_translation`
- `one_chunk_translate_text`

    ```
    def one_chunk_translate_text(
        source_lang: str, target_lang: str, source_text: str, country: str = ""
    ) -> str:
        translation_1 = one_chunk_initial_translation(
            source_lang, target_lang, source_text
        )
        
        reflection = one_chunk_reflect_on_translation(
            source_lang, target_lang, source_text, translation_1, country
        )
        translation_2 = one_chunk_improve_translation(
            source_lang, target_lang, source_text, translation_1, reflection
        )
        return translation_2
    ```

### multichunk

```
def multichunk_translation(
    source_lang, target_lang, source_text_chunks, country: str = ""
):
    translation_1_chunks = multichunk_initial_translation(
            source_lang, target_lang, source_text_chunks
        )
    
        reflection_chunks = multichunk_reflect_on_translation(
            source_lang,
            target_lang,
            source_text_chunks,
            translation_1_chunks,
            country,
        )
    
        translation_2_chunks = multichunk_improve_translation(
            source_lang,
            target_lang,
            source_text_chunks,
            translation_1_chunks,
            reflection_chunks,
        )
    
        return translation_2_chunks
```

- split chunks

In [4]:
def calculate_chunk_size(token_count: int, token_limit: int) -> int:
    """
    Calculate the chunk size based on the token count and token limit.

    Args:
        token_count (int): The total number of tokens.
        token_limit (int): The maximum number of tokens allowed per chunk.

    Returns:
        int: The calculated chunk size.

    Description:
        This function calculates the chunk size based on the given token count and token limit.
        If the token count is less than or equal to the token limit, the function returns the token count as the chunk size.
        Otherwise, it calculates the number of chunks needed to accommodate all the tokens within the token limit.
        The chunk size is determined by dividing the token limit by the number of chunks.
        If there are remaining tokens after dividing the token count by the token limit,
        the chunk size is adjusted by adding the remaining tokens divided by the number of chunks.

    Example:
        >>> calculate_chunk_size(1000, 500)
        500
        >>> calculate_chunk_size(1530, 500)
        389
        >>> calculate_chunk_size(2242, 500)
        496
    """

    if token_count <= token_limit:
        return token_count

    num_chunks = (token_count + token_limit - 1) // token_limit
    chunk_size = token_count // num_chunks

    remaining_tokens = token_count % token_limit
    if remaining_tokens > 0:
        chunk_size += remaining_tokens // num_chunks

    return chunk_size

In [5]:
(1530 + 500-1) // 500

4

In [6]:
1530 // 4

382

In [7]:
1530 % 500

30

In [9]:
382 + 30//4

389